In [6]:
import pyspark
from pyspark.sql import *
from pyspark import SparkContext, SparkConf

from pyspark.sql import *
from pyspark.sql.functions import *

import pandas as pd
import sqlite3

In [37]:
conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext(conf=conf)

spark = SparkSession.builder.getOrCreate()


In [46]:

conn = sqlite3.connect("../simple.db")

In [49]:

df_people = pd.read_sql_query("SELECT * FROM people", conn, dtype={"home_colony_id": "Int64"})

print("type: ")
print(df_people["home_colony_id"].dtype)

df_colonies = pd.read_sql_query("SELECT * FROM colonies", conn)

print("df_people")
print(df_people.head())
print()
print("df_colonies")
print(df_colonies.head())

df_people.to_csv("people.csv", index=False)
df_colonies.to_csv("colonies.csv", index=False)


type: 
float64
df_people
   id first_name last_name salutation call_sign  home_colony_id
0   1        Lee     Adama        Sir    Apollo             4.0
1   2    William     Adama        Sir    Husker             4.0
2   3       Kara    Thrace        Sir  Starbuck             4.0
3   4      Gaius    Baltar        Dr.      None             1.0
4   5      Laura    Roslin        Ms.      None             4.0

df_colonies
   id      name
0   1   Aerilon
1   2   Aquaria
2   3  Canceron
3   4   Caprica
4   5   Gemenon


In [ ]:

rdd_people = sc.parallelize(df_people.values.tolist())

rdd_people.collect()

In [29]:
bsg_people = spark.read.csv("people.csv", header=True)
bsg_colonies = spark.read.csv("colonies.csv", header=True)

bsg_data = bsg_people.join(bsg_colonies, bsg_people.home_colony_id == bsg_colonies.id)

bsg_data.show()

+---+----------+---------+----------+---------+--------------+---+----+
| id|first_name|last_name|salutation|call_sign|home_colony_id| id|name|
+---+----------+---------+----------+---------+--------------+---+----+
+---+----------+---------+----------+---------+--------------+---+----+



In [21]:
colony_counts = bsg_data.groupBy("home_colony_id").count().sort(desc("count"))

colony_counts.show()

+--------------+-----+
|home_colony_id|count|
+--------------+-----+
|          NULL|    6|
|           4.0|    5|
|          13.0|    3|
|           1.0|    1|
|           9.0|    1|
|           8.0|    1|
+--------------+-----+



In [36]:
conn.close()
sc.stop()